# Dog Breed Identification
## Farhad Navid 

### transfer learning with Aug data set

* Load data set
* Load weights from ImageNet features for pretrained VGG19 model "block4_pool".
* Run model
* Create train and test
* Run SVM model
* Record the performance. 

In [1]:
# initialization
import matplotlib.pyplot as plt 
import PIL
import pandas as pd
import os
import glob
import time
import numpy as np
import h5py
import tensorflow as tf
import deepdish as dd

from array import *

from tqdm import tqdm
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Concatenate, Dense, Dropout, Flatten, Activation, GlobalMaxPooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D 

from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
from keras import backend as K
from keras.optimizers import Adam

from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input

from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score,confusion_matrix,accuracy_score,classification_report
from sklearn.model_selection import train_test_split


%matplotlib inline
K.set_image_dim_ordering( 'tf' )

Using TensorFlow backend.


This next block of code does represent the Directory structure of the data once the repository was selected.  In this instance the [**AWS**](https://www.AWS.Amazon.com/) (paid service) were utilized.  

In [2]:
# directory structure of the data

dpath_train = '/home/ubuntu/train'      # Data path to training data set
dpath_test  = '/home/ubuntu/test'       # Data path to test data set
dpath       = '/home/ubuntu'
label_f     = '/home/ubuntu/labels.csv' # Path to the label file 

**Load the augmented data file** 

In [ ]:
# Retrive already created data set with Augmented Images

Aug_train_data = '/home/ubuntu/train_aug_data.hdf5' #
f= dd.io.load(Aug_train_data)
X_train_aug = f['X']
y_train_aug = f['Y']

In [3]:
# point to location of the Train images
images = glob.glob('/home/ubuntu/train/*.jpg')
img_data = []
# resize and add new axis to images
for i in tqdm(images):
    img = image.load_img(i, target_size=(224, 224))
    img_data.append(image.img_to_array(img)[np.newaxis, :, :, :])
# concatenate the images    
tr_img_data_np = np.concatenate(img_data, )
# save them as "unsigned Integer" for the smaller size.
tr_img_data_np = tr_img_data_np.astype('uint8')
# create the label 
labels = pd.read_csv(label_f)

100%|██████████| 10222/10222 [04:09<00:00, 41.00it/s]


In [4]:
print(tr_img_data_np.shape,tr_img_data_np.dtype)

(10222, 224, 224, 3) uint8


In [5]:
'''
    @parameters:
        dataset: the feature training dataset in numpy array with shape [num_examples, num_rows, num_cols, num_channels] (since it is an image in numpy array)
        dataset_labels: the corresponding training labels of the feature training dataset in the same order, and numpy array with shape [num_examples, <anything>]
        augmentation_factor: how many times to perform augmentation.
        use_random_rotation: whether to use random rotation. default: true
        use_random_shift: whether to use random shift. default: true
        use_random_shear: whether to use random shear. default: true
        use_random_zoom: whether to use random zoom. default: true
        
    @returns:
        augmented_image: augmented dataset
        augmented_image_labels: labels corresponding to augmented dataset in order.
        
    for the augmentation techniques documentation, go here:
        https://www.tensorflow.org/api_docs/python/tf/contrib/keras/preprocessing/image/random_rotation
        https://www.tensorflow.org/api_docs/python/tf/contrib/keras/preprocessing/image/random_shear
        https://www.tensorflow.org/api_docs/python/tf/contrib/keras/preprocessing/image/random_shift
        https://www.tensorflow.org/api_docs/python/tf/contrib/keras/preprocessing/image/random_zoom
'''
def augment_data(dataset, dataset_labels, augementation_factor=1, use_random_rotation=True, use_random_shear=True, use_random_shift=True, use_random_zoom=True):
    augmented_image = []
    augmented_image_labels = []

    for num in tqdm(range(0, dataset.shape[0])):  # tqdm progress bar
        for i in range(0, augementation_factor):
            # original image:
            augmented_image.append(dataset[num])
            augmented_image_labels.append(dataset_labels[num])

            if use_random_rotation:
                
                augmented_image.append(tf.contrib.keras.preprocessing.image.random_rotation(dataset[num], 20, row_axis=0, col_axis=1, channel_axis=2))
                augmented_image_labels.append(dataset_labels[num])

            if use_random_shear:
                augmented_image.append(tf.contrib.keras.preprocessing.image.random_shear(dataset[num], 0.2, row_axis=0, col_axis=1, channel_axis=2))
                augmented_image_labels.append(dataset_labels[num])

            if use_random_shift:
                augmented_image.append(tf.contrib.keras.preprocessing.image.random_shift(dataset[num], 0.2, 0.2, row_axis=0, col_axis=1, channel_axis=2))
                augmented_image_labels.append(dataset_labels[num])

            if use_random_zoom:
                # update: zoomrange (second arg) should be tuple of floats
                augmented_image.append(tf.contrib.keras.preprocessing.image.random_zoom(dataset[num], (0.9, 0.9), row_axis=0, col_axis=1, channel_axis=2))
                augmented_image_labels.append(dataset_labels[num])

    return np.array(augmented_image), np.array(augmented_image_labels)

In [6]:
# Augment train data set this will increase the data set size by 5X
aug_images, aug_labels = augment_data(tr_img_data_np, labels.breed)

100%|██████████| 10222/10222 [06:55<00:00, 24.59it/s]


In [7]:
# check point make sure the size and shape are as expected.
print(aug_images.shape,aug_labels.shape,aug_images.dtype)
# Make sure the data type is expected.
aug_images.dtype

(51110, 224, 224, 3) (51110,) uint8


dtype('uint8')

In [8]:
# Now let's make sure we have all the unique dog names identified and chk the total number.

unique_Dog_Breed = []
for i in aug_labels:
    if i not in unique_Dog_Breed:
        unique_Dog_Breed.append(i)
    
unique_Dog_Breed.sort()   # Now sort the unique lable alphabatically. Next we need to count howmany unique label

count = 0
unique_Dog_Breed_Num = []   # integer assiciated with every unique Dog name

# with the next loops we are creating integers for each unique labels. in prepration of catagorizing the Y.

for i in unique_Dog_Breed:
    unique_Dog_Breed_Num.append([i, count])
    count += 1

for i in range(len(aug_labels)):
    for j in unique_Dog_Breed_Num:
        if aug_labels[i] == j[0]:
            aug_labels[i] = j[1]
            break

print("unique Dog Breed Numbers = ",len(unique_Dog_Breed_Num))
y_train = to_categorical(aug_labels)

unique Dog Breed Numbers =  120


In [9]:
print(y_train.shape, y_train.dtype)

(51110, 120) float32


In [13]:
# now save to hdf5 

#os.chdir(dpath)
with h5py.File('traindata_aug.hdf5','w') as f:
    f.create_dataset('X', data=aug_images)
    f.create_dataset('Y', data=y_train)
# # Save the X_train and y_train to a HD5 file with Deep Dish. 
# import deepdish as dd
# Aug_train_data = '/home/ubuntu/train_aug_data.hdf5'
# dd.io.save(Aug_train_data, {'X': aug_images, 'Y': aug_labels}, compression=('blosc', 9))

In [14]:
X_train_aug = aug_images
y_train_aug = aug_labels

In [15]:
print(X_train_aug.shape,X_train_aug.dtype,y_train_aug.shape,y_train_aug.dtype)

(51110, 224, 224, 3) uint8 (51110,) <U30


### Notes
**Only 10000 images of the 51110 images are used**

Due to the size of the input file and the number of features for the **block4_pool** could not run the model completely. A different strategy is required to run the full model and that was beyond the time scope of this project.  
following is the example of max number of images we could process without getting the memory error.


In [16]:
X_train_s = X_train_aug[:10000] 
y_train_s = y_train_aug[:10000]

## Transfer Learning

In [17]:
# transfer learning
# Get the weights from imagenet for the VGG19 model
base_model = VGG19(weights='imagenet')
# now select the layer to get the features from in this case "block4_pool" was selected
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

# Create a zero Numpy array with the shape of np.zero((Xs_train.shape[0],(block4_pool_features))) 
# for this modle the (image#,14,14,512)

train_set = np.zeros((X_train_s.shape[0],14,14,512))

## This section of code was written to findout the dimmention of the Block4_pool_features.

# x = preprocess_input(X_train[1])  # get one sample of X_train
# print(x.shape)                    # Chk the shape 
# x = np.expand_dims(x,axis=0)      # Ad the image num to the list
# print(x.shape)                    # Check the shape

# block4_pool_features_org = model.predict(x)  # Create one entry to see the shape

# this loop will fill the train_set numpy array (each x_train runs through model with extracted VGG features)
for i in tqdm(range(X_train_s.shape[0])):
#    K.clear_session()
    x = preprocess_input(X_train_s[i])
    x = np.expand_dims(x,axis=0)
    block4_pool_features = model.predict(x)
    train_set[i]=block4_pool_features


100%|██████████| 10000/10000 [04:50<00:00, 34.46it/s]


In [18]:
block4_pool_features.shape

(1, 14, 14, 512)

### Create Train and Test data set for SVM model

In [19]:
# we split training data in to train and test. 

train_set = train_set.reshape(train_set.shape[0],-1)
X= train_set
y= y_train_s

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

## SVM model
* **Model Fit**
* **Model Prediction**

In [20]:
# Run the svm 

t0=time.time()
clf = SVC()
clf.fit(X_train, y_train)
t1=time.time()
print(t1-t0,"seconds")

# predict 
t0=time.time()
pred_Train=clf.predict(X_train) 
pred_Test=clf.predict(X_test) 
t1=time.time()
print(t1-t0,"seconds")
# print(pred_Train[:10],pred_Test[:10])
# print(clf.predict([train_set[5]]))

22428.387664556503 seconds
12288.37120962143 seconds


## Performance
* Cohen's Kappa Score
* Accuracy score
* classification report
* F1 Score
* confusion matrix (This matrix was not selected since the entire model did not run and result was not as interesting)

In [21]:
print(cohen_kappa_score(y_train, pred_Train),'Train data set cohen kappa score')
print(cohen_kappa_score(y_test, pred_Test),'Test data set cohen kappa score')

#confusion_matrix(y_train, pred_Train)

print(accuracy_score(y_train, pred_Train),'Accuracy Score Train Data set')
print(accuracy_score(y_test, pred_Test), 'Accuracy Score Test Data set')

print(classification_report(y_train, pred_Train))
print(classification_report(y_test, pred_Test))

print(metrics.f1_score(y_train, pred_Train,average='weighted'),'matrics f1 score train data set')
print(metrics.f1_score(y_test, pred_Test,average='weighted'),'matrics f1 score test data set')

0.9997758066476816 Train data set cohen kappa score
0.00909573426813326 Test data set cohen kappa score
0.9997777777777778 Accuracy Score Train Data set
0.018 Accuracy Score Test Data set
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        81
          1       1.00      1.00      1.00        95
         10       1.00      1.00      1.00       113
        100       1.00      1.00      1.00        89
        101       1.00      1.00      1.00        82
        102       1.00      1.00      1.00        96
        103       1.00      1.00      1.00        40
        104       1.00      1.00      1.00        59
        105       1.00      1.00      1.00        83
        106       1.00      1.00      1.00        64
        107       1.00      1.00      1.00        65
        108       1.00      1.00      1.00        67
        109       1.00      1.00      1.00        74
         11       1.00      1.00      1.00       115
        110     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
